In [ ]:
# Winter Sunlight Accessibility – Multi-Hour Scenarios
## Berlin-Mitte | 21 December (Worst-case winter day)

'''This notebook evaluates winter sunlight exposure for public spaces and benches
in Berlin-Mitte at discrete hourly scenarios (10:00, 12:00, 14:00).

The analysis is scenario-based and intentionally avoids real-time simulation'''


In [ ]:
import os
import numpy as np
import geopandas as gpd
from shapely.affinity import translate
import folium


In [ ]:
os.chdir(r"C:\Users\romak\Desktop\Urban_Technology_Berlin_Mitte")


In [ ]:
# Buildings with height (uniform height model)
buildings = gpd.read_file("data/processed/buildings_with_height.geojson")

# Public spaces & benches (OSM-derived)
public_spaces = gpd.read_file("data/processed/mitte_public_spaces.geojson")

# Berlin-Mitte boundary
mitte = gpd.read_file("maps/mitte.geojson").to_crs(epsg=25833)


In [ ]:
buildings.shape, public_spaces.shape


In [ ]:
# Approximate sun positions for Berlin on 21 December
sun_scenarios = {
    "10:00": {"altitude": 7.5,  "azimuth": 145},
    "12:00": {"altitude": 14.1, "azimuth": 179},
    "14:00": {"altitude": 7.0,  "azimuth": 215}
}


In [ ]:
def compute_sun_shadow(buildings, public_spaces, altitude, azimuth):

    shadow_length = buildings["height_m"] / np.tan(np.deg2rad(altitude))
    shadow_angle = np.deg2rad(azimuth + 180)

    dx = shadow_length * np.sin(shadow_angle)
    dy = shadow_length * np.cos(shadow_angle)

    shadows = buildings.copy()
    shadows["geometry"] = [
        translate(geom, xoff=dx_i, yoff=dy_i)
        for geom, dx_i, dy_i in zip(buildings.geometry, dx, dy)
    ]

    shadows = shadows[shadows.is_valid]
    public_spaces = public_spaces[public_spaces.is_valid]

    shaded = gpd.sjoin(
        public_spaces,
        shadows[["geometry"]],
        how="left",
        predicate="intersects"
    )

    shaded["is_shaded"] = shaded.index_right.notnull()
    shaded["sun_status"] = shaded["is_shaded"].map(
        {True: "Shaded", False: "Sunny"}
    )

    return shaded


In [ ]:
hourly_spaces = {}
hourly_benches = {}

for hour, params in sun_scenarios.items():

    shaded = compute_sun_shadow(
        buildings,
        public_spaces,
        params["altitude"],
        params["azimuth"]
    )

    # Public spaces (polygons)
    spaces = shaded[
        shaded.geometry.type.isin(["Polygon", "MultiPolygon"])
    ].copy()

    spaces["display_name"] = spaces["name"].fillna("Unnamed public space")
    hourly_spaces[hour] = spaces.to_crs(epsg=4326)

    # Benches (points)
    benches = shaded[
        (shaded["amenity"] == "bench") &
        (shaded.geometry.type == "Point")
    ].copy()

    benches["display_name"] = "Bench"
    hourly_benches[hour] = benches.to_crs(epsg=4326)


In [ ]:
{h: len(v) for h, v in hourly_spaces.items()}


In [ ]:
m = folium.Map(
    location=[52.52, 13.405],
    zoom_start=12,
    tiles="CartoDB positron",
    control_scale=True
)

bounds = mitte.to_crs(epsg=4326).total_bounds
m.fit_bounds([[bounds[1], bounds[0]], [bounds[3], bounds[2]]])


In [ ]:
def space_style(feature):
    return {
        "fillColor": "#fdd835" if feature["properties"]["sun_status"] == "Sunny" else "#b0bec5",
        "color": "none",
        "fillOpacity": 0.75
    }


In [ ]:
for hour, gdf in hourly_spaces.items():

    fg = folium.FeatureGroup(
        name=f"Public spaces – {hour}",
        show=True if hour == "12:00" else False
    )

    folium.GeoJson(
        gdf,
        style_function=space_style,
        tooltip=folium.GeoJsonTooltip(
            fields=["display_name", "sun_status"],
            aliases=["Public space:", f"Sun at {hour}:"],
            sticky=True
        )
    ).add_to(fg)

    fg.add_to(m)


In [ ]:
for hour, gdf in hourly_benches.items():

    fg = folium.FeatureGroup(
        name=f"Benches – {hour}",
        show=False
    )

    for _, row in gdf.iterrows():
        folium.CircleMarker(
            location=[row.geometry.y, row.geometry.x],
            radius=4,
            color="#fbc02d" if row["sun_status"] == "Sunny" else "#78909c",
            fill=True,
            fill_opacity=0.9,
            tooltip=f"Bench — {row['sun_status']} at {hour}"
        ).add_to(fg)

    fg.add_to(m)


In [ ]:
for hour, gdf in hourly_spaces.items():
    fg = folium.FeatureGroup(name=f"Public spaces – {hour}", show=False)

    folium.GeoJson(
        gdf,
        style_function=space_style,
        tooltip=folium.GeoJsonTooltip(
            fields=["display_name", "sun_status"],
            aliases=["Public space:", f"Sun at {hour}:"],
            sticky=True
        )
    ).add_to(fg)

    fg.add_to(m)


In [ ]:
folium.GeoJson(
    mitte.to_crs(epsg=4326).boundary,
    name="Berlin-Mitte",
    style_function=lambda x: {"color": "black", "weight": 3}
).add_to(m)


In [ ]:
folium.LayerControl(collapsed=False).add_to(m)
m


In [ ]:
#### Questions Analysis #####

In [ ]:
# Which public spaces are actually suitable for winter use?

In [ ]:
import pandas as pd
import geopandas as gpd

all_hours = []

for hour, gdf in hourly_spaces.items():
    temp = gdf.copy()
    temp["hour"] = hour
    all_hours.append(temp)

all_hours = gpd.GeoDataFrame(
    pd.concat(all_hours, ignore_index=True),
    crs=hourly_spaces["12:00"].crs
)

In [ ]:
# create a unique ID per space
all_hours["space_uid"] = all_hours.groupby(all_hours.geometry.apply(id)).ngroup()

suitability = (
    all_hours
    .groupby("space_uid")["sun_status"]
    .apply(lambda x: (x == "Sunny").sum())
    .reset_index(name="sunny_hours")
)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
suitability["sunny_hours"].value_counts().sort_index().plot(kind="bar")
plt.xlabel("Number of sunny hours")
plt.ylabel("Number of public spaces")
plt.title("Winter Suitability of Public Spaces")
plt.show()

In [ ]:
suitability["sunny_hours"].value_counts().sort_index()

In [ ]:
#What share of benches receive any winter sunlight?

In [ ]:

# Filter benches
benches = all_hours[all_hours["amenity"] == "bench"]

# Percentage of sunny vs shaded benches
bench_stats = (
    benches["sun_status"]
    .value_counts(normalize=True) * 100
)

# Plot pie chart
bench_stats.plot(
    kind="pie",
    autopct="%1.1f%%",
    startangle=90,
    colors=["#b0bec5", "#fdd835"],
    ylabel=""
)
plt.title("Winter Sun Exposure of Benches")
plt.show()

In [ ]:
### Urban furniture placement appears to prioritise shade (good for summer) but unintentionally reduces winter usability.

In [ ]:
#Which important public spaces in Berlin-Mitte provide the least winter sunlight for seating?

In [ ]:
# Select 12:00 scenario explicitly
spaces_12 = hourly_spaces["12:00"].copy()
benches_12 = hourly_benches["12:00"].copy()

In [ ]:
spaces_12 = spaces_12.to_crs(epsg=25833)
benches_12 = benches_12.to_crs(epsg=25833)

In [ ]:
# Clean up previous spatial join artifacts
for col in ["index_right", "index_left"]:
    if col in benches_12.columns:
        benches_12 = benches_12.drop(columns=col)
    if col in spaces_12.columns:
        spaces_12 = spaces_12.drop(columns=col)

In [ ]:
benches_with_space = gpd.sjoin(
    benches_12,
    spaces_12[["name", "geometry"]],
    how="left",
    predicate="within"
)

In [ ]:
benches_with_space.columns

In [ ]:
benches_with_space.columns

In [ ]:
benches_with_space = benches_with_space.rename(
    columns={"name_right": "public_space"}
)

In [ ]:
benches_with_space["public_space"] = benches_with_space["public_space"].fillna(
    "Outside named public space"
)

In [ ]:
benches_with_space[["public_space", "sun_status"]].head(10)

In [ ]:
bench_loss = (
    benches_with_space
    .groupby("public_space")
    .agg(
        sunny=("sun_status", lambda x: (x == "Sunny").sum()),
        shaded=("sun_status", lambda x: (x == "Shaded").sum())
    )
    .reset_index()
)

bench_loss["sun_ratio"] = (
    bench_loss["sunny"] /
    (bench_loss["sunny"] + bench_loss["shaded"])
)

bench_loss.sort_values("sun_ratio").head(10)

In [ ]:
import matplotlib.pyplot as plt

worst = bench_loss.sort_values("sun_ratio").head(10)

plt.figure(figsize=(8,4))
plt.barh(
    worst["public_space"],
    worst["sun_ratio"],
)
plt.xlabel("Share of sunny benches")
plt.title("Public spaces with lowest winter sunlight access")
plt.gca().invert_yaxis()
plt.show()

In [ ]:
#Campus-Scale Bench Sunlight Analysis

In [ ]:
# BHT main campus location (Wedding, Berlin-Mitte)
from shapely.geometry import Point

bht_point = Point(13.3536, 52.5431)  # lon, lat
bht = gpd.GeoDataFrame(
    {"name": ["Berliner Hochschule für Technik"]},
    geometry=[bht_point],
    crs="EPSG:4326"
).to_crs(epsg=25833)

# Create a campus buffer (300m walking-scale radius)
bht_buffer = bht.buffer(300)

In [ ]:
# Ensure benches are in projected CRS
benches_12 = benches_12.to_crs(epsg=25833)

# Select benches within campus buffer
bht_benches = benches_12[
    benches_12.geometry.within(bht_buffer.iloc[0])
].copy()

len(bht_benches)

In [ ]:
bht_stats = (
    bht_benches["sun_status"]
    .value_counts()
    .rename_axis("sun_status")
    .reset_index(name="count")
)

bht_stats["percentage"] = (
    bht_stats["count"] / bht_stats["count"].sum() * 100
)

bht_stats

In [ ]:
colors = {
    "Sunny": "#fdd835",
    "Shaded": "#b0bec5"
}

plt.figure(figsize=(6,6))
plt.pie(
    bht_stats["percentage"],
    labels=bht_stats["sun_status"],
    autopct="%.1f%%",
    startangle=90,
    colors=[colors[s] for s in bht_stats["sun_status"]]
)
plt.title("Winter Sun Exposure of Benches\nBerliner Hochschule für Technik")
plt.show()